## 에이전트를 Function 도구로 사용하기

이 과정을 통해서 에이전트를 Function 도구로 사용하는 방법을 보여줍니다. 이를 통해 한 에이전트가 다른 에이전트를 도구로 호출할 수 있습니다.

#### 에이전트를 Function 도구로 생성 및 사용하기.

에이전트를 .as_tool()로 호출하여 다른 에이전트에 도구로 제공함으로써 ChatAgent를 Function 도구로 사용할 수 있습니다. 이를 통해 에이전트를 조합하고 더욱 고급 워크플로우를 구축할 수 있습니다. 먼저, 에이전트가 사용할 Function 도구를 생성하고 이를 함수로 노출합니다. 그리고, Function 도구로 사용할 ChatAgent 를 생성합니다.

In [ ]:
from typing import Annotated
from pydantic import Field

## 날씨 Function 도구 정의
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C."

## 날씨 에이전트 생성
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

weather_agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="WeatherAgent",
    description="An agent that answers questions about the weather.",
    instructions="You answer questions about the weather.",
    tools=get_weather
)

이제, 메인 에이전트를 생성하고 weather_agent를 Function 도구로 변환하는 .as_tool() 함수를 호출하여 weather_agent를 Function 도구로 제공하고 실행해 봅니다.

In [ ]:
main_agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="You are a helpful assistant who responds in korean.",
    tools=weather_agent.as_tool()
)

result = await main_agent.run("What is the weather like in Seoul?")
print(result.text)

에이전트를 도구로 변환할 때 도구 이름, 설명 및 인수 이름을 사용자 지정할 수도 있습니다.

In [ ]:
# Convert agent to tool with custom parameters
weather_tool = weather_agent.as_tool(
    name="WeatherLookup",
    description="Look up weather information for any location",
    arg_name="query",
    arg_description="The weather query or location"
)

main_agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="You are a helpful assistant who responds in Korean.",
    tools=weather_tool
)

result = await main_agent.run("What is the weather like in Jeju?")
print(result.text)